In [1]:
# Daten einlesen
import pandas as pd

df = pd.read_csv('HateSpeechDataset.csv')

# erste Zeilen
print(df.head())

# Spalten
print(df.columns)








                                             Content Label  \
0  denial of normal the con be asked to comment o...     1   
1  just by being able to tweet this insufferable ...     1   
2  that is retarded you too cute to be single tha...     1   
3  thought of a real badass mongol style declarat...     1   
4                                afro american basho     1   

                                         Content_int  
0  [146715, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,...  
1  [146715, 14, 15, 16, 17, 7, 18, 19, 20, 21, 22...  
2  [146715, 28, 29, 30, 26, 31, 32, 7, 5, 33, 28,...  
3  [146715, 35, 1, 24, 36, 37, 38, 39, 40, 1, 41,...  
4                       [146715, 46, 47, 48, 146714]  
